In [1]:
import numpy as np 
import os
from glob import glob
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import keras
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D,Dense, Flatten

C:\Progra_Files\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
DATA_PATH = "./Dataset/leapGestRecog/"
img_dims = (32, 32)

lookup = dict()
reverselookup = dict()
count = 0

for j in os.listdir(DATA_PATH+"00/"):
    if not j.startswith('.'):
        lookup[j] = count	
        reverselookup[count] = j
        count = count + 1

In [4]:
reverselookup

{0: '01_palm',
 1: '02_l',
 2: '03_fist',
 3: '04_fist_moved',
 4: '05_thumb',
 5: '06_index',
 6: '07_ok',
 7: '08_palm_moved',
 8: '09_c',
 9: '10_down'}

In [3]:
x_data = []
y_data = []
datacount = 0 
print ("::: Gesture dictionaries created.")

for dir in glob(DATA_PATH + "*"):
	for subdir in glob(dir + "/*"):
		count = 0
		for image in glob(subdir + "/*.png"):
			img = Image.open(image).convert('L')
			img = img.resize((320, 120))
			image_array = np.array(img)
			x_data.append(image_array)
			count += 1
		y = np.full((count, 1), int(lookup[subdir.split("\\")[-1]]))
		y_data.append(y)
		datacount += count
		
		
x_data = np.array(x_data, dtype = 'float32').reshape((datacount, 120, 320, 1))/255
y_data = np.array(y_data).reshape(datacount, 1)
print("::: Images successfully loaded.")
print("Total number of images =", len(x_data))

y_data = to_categorical(y_data)

print("::: LABELS successfully loaded.")

::: Gesture dictionaries created.
::: Images successfully loaded.
Total number of images = 20000
::: LABELS successfully loaded.


In [4]:
x_train,x,y_train,y = train_test_split(x_data,y_data,test_size = 0.3)
x_validate,x_test,y_validate,y_test = train_test_split(x,y,test_size = 0.5)

print("::: TRAINING SET has been created.")
print("::: VALIDATION SET has been created.")
print("::: TESTING SET has been created.")

::: TRAINING SET has been created.
::: VALIDATION SET has been created.
::: TESTING SET has been created.


In [9]:
model = Sequential()
model.add(Conv2D(32, (5, 5), strides=(2, 2), activation='relu', input_shape=(120, 320,1))) 
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu')) 
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(10, activation='softmax'))

print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 58, 158, 32)       832       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 29, 79, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 27, 77, 64)        18496     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 13, 38, 64)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 11, 36, 64)        36928     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 5, 18, 64)         0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 5760)              0         
__________

In [10]:
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x_train, 
          y_train, 
          epochs=10, 
          batch_size=64, 
          validation_data=(x_validate, y_validate))

Instructions for updating:
Use tf.cast instead.
Train on 14000 samples, validate on 3000 samples
Epoch 1/10
14000/14000 [==============================] - 329s 24ms/step - loss: 0.3574 - acc: 0.8879 - val_loss: 0.0381 - val_acc: 0.9887
Epoch 2/10
14000/14000 [==============================] - 222s 16ms/step - loss: 0.0151 - acc: 0.9956 - val_loss: 0.0018 - val_acc: 0.9997
Epoch 3/10
14000/14000 [==============================] - 223s 16ms/step - loss: 0.0114 - acc: 0.9977 - val_loss: 0.0054 - val_acc: 0.9993
Epoch 4/10
14000/14000 [==============================] - 221s 16ms/step - loss: 0.0034 - acc: 0.9994 - val_loss: 0.0025 - val_acc: 0.9993
Epoch 5/10
14000/14000 [==============================] - 221s 16ms/step - loss: 0.0030 - acc: 0.9992 - val_loss: 0.0061 - val_acc: 0.9980
Epoch 6/10
14000/14000 [==============================] - 221s 16ms/step - loss: 0.0015 - acc: 0.9996 - val_loss: 4.3152e-04 - val_acc: 0.9997
Epoch 7/10
14000/14000 [==============================] - 221s 16

In [11]:
[loss, acc] = model.evaluate(x_test,y_test,verbose=1)
print("Accuracy:" + str(acc*100))

3000/3000 [==============================] - 23s 8ms/step
Accuracy:99.93333333333332


In [13]:
from keras.models import model_from_json

model_json = model.to_json()
with open("./Models/GestureDetectionModel.json", "w") as json_file:
    json_file.write(model_json)
 # serialize weights to HDF5
model.save_weights("./Models/GestureDetectionModel_weights.h5")
print("Saved model to disk")


Saved model to disk
